In [1]:
from urllib.request import Request, urlopen, addinfourl
from bs4 import BeautifulSoup
import re
import csv
import time

def writeinfile(listF):
    '''
        Takes contents list as argument
        and appends to the csv file
    '''
    csvFile = open('kk.csv', 'a+')
    try:
        writer = csv.writer(csvFile)
        writer.writerow(('Title','Place','Land Size','PublishDate','Price')) # Content title
        for i in range(0, len(listF), 6):
            writer.writerow((listF[i], listF[i+1], listF[i+2], listF[i+3], listF[i+4]))
    finally:
        csvFile.close()
    
def getContents(pageUrl):
    '''
        Takes url of the webpage as argument and
        returns list of contents parsed from that url.
    '''
    
    request = Request(f"https://hamrobazaar.com/search.php{pageUrl}")
    request.add_header("User-Agent", "*") # to prevent HttpError, User-agent is to be defined.

    html = urlopen(request)
    bs = BeautifulSoup(html.read(), 'lxml')
    
    content_list = list() # list to store contents ie, title, place, etc.
    
    for table in bs.find_all('table'):
        if len(list(table.tr.children)) == 13:
            if table.center:
                contents = table.tr
                titles = contents.find_all('font',style='font-size:13px;')
                place_landsize = contents.find_all('font',color='#565d60')
                post_date = contents.find_all('td',width='85')
                prices = contents.find_all('td',width='100')
                
                # add title, place, landsize, postdate, prices to content_list first then append that list to csv file.
                for title in titles:
                    content_list.append(title.b.text)          
                    
                for psize in place_landsize:
                    totaltext = psize.b.i.text
                    textlist = totaltext.split('|')
                    if len(textlist) == 0:
                        landSize = 0
                        place = " "
                    elif len(textlist) == 1:
                        place = textlist[0]
                        landSize = " "
                    else:
                        place = textlist[0]
                        landSize = textlist[-1]
                    content_list.append(place)
                    content_list.append(landSize)
                    
                for pdate in post_date:
                    content_list.append(pdate.text)
                    
                for price in prices:
                    content_list.append(price.b.text)
                
                content_list.append("\n")
    return content_list
      
pages = list() # all links to be traversed.

def getLinks(pageUrl):
    '''
        Takes pageUrl
        and retrieves the links of other pages
        recursively.
    '''
    global pages
    request = Request("https://hamrobazaar.com/search.php?do_search=Search&searchword=land")
    request.add_header("User-Agent", "*")

    html = urlopen(request)
    bs = BeautifulSoup(html.read(), 'lxml')
    
    for link in bs.find_all('a', href=re.compile('(offset=\d+)$')):
        if 'href' in link.attrs:
            if link['href'] not in pages:
                # We have encountered a new page
                newPage = link['href']
                print(newPage)
                pages.append(newPage)
                getLinks(newPage)
                
# main control flow  

getLinks('')

# if first page has link "....offset=20", 2nd page has link "offset=40", incremented by 20.
for i in range(20, int(pages[-1].split('=')[-1]) + 1, 20):
    link = f"?&do_search=1&searchword=land&order=popularad&offset={i}"
    if link not in pages:
        pages.append(link)


for page in pages:
    contentList = getContents(page) 
    writeinfile(contentList) # appends contents in csv file
    print("Delay"*20) # message before sleep
    time.sleep(25) # prevent server overload

?&do_search=1&searchword=land&order=popularad&offset=20
?&do_search=1&searchword=land&order=popularad&offset=40
?&do_search=1&searchword=land&order=popularad&offset=60
?&do_search=1&searchword=land&order=popularad&offset=80
?&do_search=1&searchword=land&order=popularad&offset=100
?&do_search=1&searchword=land&order=popularad&offset=120
?&do_search=1&searchword=land&order=popularad&offset=140
?&do_search=1&searchword=land&order=popularad&offset=160
?&do_search=1&searchword=land&order=popularad&offset=180
?&do_search=1&searchword=land&order=popularad&offset=200
?&do_search=1&searchword=land&order=popularad&offset=1780
?&do_search=1&searchword=land&order=popularad&offset=3780
?&do_search=1&searchword=land&order=popularad&offset=5580
DelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelay
DelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelayDelay


KeyboardInterrupt: 